# Импорты

In [7]:
import numpy as np
from scipy.stats import norm
import networkx as nx

# Класс сетевого анализа

In [8]:
class NetworkPlanning:
    def __init__(self):
        self.works = {}
        self.graph = nx.DiGraph()
        
    def add_work(self, name, dependencies, t_pes, t_ver, t_opt, cost_reduction=None):
        """Добавление работы с её характеристиками"""
        self.works[name] = {
            'dependencies': dependencies,
            't_pes': t_pes,
            't_ver': t_ver,
            't_opt': t_opt,
            'cost_reduction': cost_reduction
        }
    
    def calculate_three_param_model(self):
        """Расчет для трехпараметрической модели"""
        results = {}
        for name, work in self.works.items():
            t_oj = (work['t_pes'] + 4 * work['t_ver'] + work['t_opt']) / 6
            variance = ((work['t_pes'] - work['t_opt']) / 6) ** 2
            
            results[name] = {
                't_oj': round(t_oj),
                'variance': round(variance, 2)
            }
        return results
    
    def calculate_two_param_model(self):
        """Расчет для двухпараметрической модели"""
        results = {}
        for name, work in self.works.items():
            t_oj_star = (3 * work['t_pes'] + 2 * work['t_opt']) / 5
            variance = ((work['t_pes'] - work['t_opt']) / 6) ** 2
            
            results[name] = {
                't_oj_star': round(t_oj_star),
                'variance': round(variance, 2)
            }
        return results
    
    def build_network(self, model_type='three_param'):
        """Построение сетевого графика"""
        self.graph.clear()
        
        if model_type == 'three_param':
            model_results = self.calculate_three_param_model()
            time_key = 't_oj'
        else:
            model_results = self.calculate_two_param_model()
            time_key = 't_oj_star'
        
        # Создаем узлы на основе зависимостей
        node_counter = 0
        event_nodes = {}
        
        # Начальное событие
        event_nodes['start'] = node_counter
        node_counter += 1
        
        # Создаем узлы для каждой работы
        for name, work in self.works.items():
            if work['dependencies'] == ['-']:
                start_event = 'start'
            else:
                # Для упрощения создаем узел после зависимостей
                start_event = f"after_{'_'.join(work['dependencies'])}"
            
            end_event = f"end_{name}"
            
            if start_event not in event_nodes:
                event_nodes[start_event] = node_counter
                node_counter += 1
            if end_event not in event_nodes:
                event_nodes[end_event] = node_counter
                node_counter += 1
            
            self.graph.add_edge(
                event_nodes[start_event], 
                event_nodes[end_event], 
                weight=model_results[name][time_key],
                variance=model_results[name]['variance'],
                name=name
            )
        
        return self.graph
    
    def find_critical_path(self, model_type='three_param'):
        """Нахождение критического пути"""
        graph = self.build_network(model_type)
        
        try:
            # Находим самый длинный путь (критический путь)
            longest_path = nx.dag_longest_path(graph, weight='weight')
            critical_time = nx.dag_longest_path_length(graph, weight='weight')
            
            # Находим работы на критическом пути
            critical_works = []
            total_variance = 0
            
            for i in range(len(longest_path) - 1):
                edge_data = graph[longest_path[i]][longest_path[i+1]]
                critical_works.append(edge_data['name'])
                total_variance += edge_data['variance']
            
            sigma = np.sqrt(total_variance)
            
            return {
                'critical_time': critical_time,
                'critical_works': critical_works,
                'total_variance': total_variance,
                'sigma': sigma
            }
        except nx.NetworkXError:
            return None
    
    def calculate_probability(self, T_dir, model_type='three_param'):
        """Расчет вероятности выполнения проекта к заданному сроку"""
        critical_path = self.find_critical_path(model_type)
        if not critical_path:
            return None
        
        T_kr = critical_path['critical_time']
        sigma = critical_path['sigma']
        
        z = (T_dir - T_kr) / sigma
        laplas_func_aproctimation = 0.5
        probability = laplas_func_aproctimation + norm.cdf(z) - laplas_func_aproctimation
        
        return probability
    
    def guaranteed_interval(self, P=0.9973, model_type='three_param'):
        """Интервал гарантированного времени выполнения"""
        critical_path = self.find_critical_path(model_type)
        if not critical_path:
            return None
        
        T_kr = critical_path['critical_time']
        sigma = critical_path['sigma']
        
        if P == 0.9973:
            delta = 3 * sigma
        else:
            z = norm.ppf((1 + P) / 2)
            delta = z * sigma
        
        return (T_kr - delta, T_kr + delta)
    
    def max_duration_with_reliability(self, gamma=0.95, model_type='three_param'):
        """Максимальный срок выполнения с заданной надежностью"""
        critical_path = self.find_critical_path(model_type)
        if not critical_path:
            return None
        
        T_kr = critical_path['critical_time']
        sigma = critical_path['sigma']
        
        z_gamma = norm.ppf(gamma)
        T_max = T_kr + z_gamma * sigma
        
        return T_max

# Основные расчеты

In [9]:
# Создаем экземпляр класса
np_solver = NetworkPlanning()

# Данные из варианта 5 (с вашего скриншота)
works_data = [
    # name, dependencies, t_pes, t_ver, t_opt, cost_reduction
    ('b1', ['-'], 9, 4, 3, 3),
    ('b2', ['-'], 7, 5, 4, 7),
    ('b3', ['-'], 13, 6, 2, 5),
    ('b4', ['b1'], 8, 6, 3, 8),
    ('b5', ['b2'], 6, 5, 2, 10),
    ('b6', ['b2'], 10, 8, 3, 2),
    ('b7', ['b3'], 9, 4, 3, 6),
    ('b8', ['b4', 'b5'], 13, 7, 5, 4),
    ('b9', ['b6', 'b7'], 9, 6, 2, 8),
    ('b10', ['b6', 'b7', 'b8'], 11, 5, 3, 3),
    ('b11', ['b9'], 9, 5, 2, 5)
]

for work in works_data:
    np_solver.add_work(*work)

T_dir = 25  # Директивный срок
gamma = 0.9  # Надежность

print("=== ТРЕХПАРАМЕТРИЧЕСКАЯ МОДЕЛЬ ===")

# Расчет ожидаемого времени и дисперсии
three_param = np_solver.calculate_three_param_model()
print("\nОжидаемое время и дисперсия (трехпараметрическая модель):")
for work, values in three_param.items():
    print(f"{work}: t_ож = {values['t_oj']}, D(x) = {values['variance']}")

# Критический путь
critical_3param = np_solver.find_critical_path('three_param')
if critical_3param:
    print(f"\nКритический путь: {critical_3param['critical_works']}")
    print(f"Ожидаемое время выполнения: {critical_3param['critical_time']} дней")
    print(f"Дисперсия критического пути: {critical_3param['total_variance']:.2f}")
    print(f"Среднеквадратическое отклонение: {critical_3param['sigma']:.2f}")
    
    # Вероятность выполнения к сроку
    prob_3param = np_solver.calculate_probability(T_dir, 'three_param')
    print(f"\nВероятность выполнения к сроку {T_dir} дней: {prob_3param:.2%}")
    
    # Интервал гарантированного выполнения
    interval_3param = np_solver.guaranteed_interval(0.9973, 'three_param')
    print(f"Интервал гарантированного выполнения (P=0.9973): {interval_3param[0]:.1f} - {interval_3param[1]:.1f} дней")
    
    # Максимальный срок с надежностью
    max_duration_3param = np_solver.max_duration_with_reliability(gamma, 'three_param')
    print(f"Максимальный срок с надежностью {gamma}: {max_duration_3param:.1f} дней")

print("\n=== ДВУХПАРАМЕТРИЧЕСКАЯ МОДЕЛЬ ===")

# Расчет для двухпараметрической модели
two_param = np_solver.calculate_two_param_model()
print("\nОжидаемое время и дисперсия (двухпараметрическая модель):")
for work, values in two_param.items():
    print(f"{work}: t*_ож = {values['t_oj_star']}, D(x) = {values['variance']}")

# Критический путь
critical_2param = np_solver.find_critical_path('two_param')
if critical_2param:
    print(f"\nКритический путь: {critical_2param['critical_works']}")
    print(f"Ожидаемое время выполнения: {critical_2param['critical_time']} дней")
    
    # Вероятность выполнения к сроку
    prob_2param = np_solver.calculate_probability(T_dir, 'two_param')
    print(f"\nВероятность выполнения к сроку {T_dir} дней: {prob_2param:.2%}")
    
    # Интервал гарантированного выполнения
    interval_2param = np_solver.guaranteed_interval(0.9973, 'two_param')
    print(f"Интервал гарантированного выполнения (P=0.9973): {interval_2param[0]:.1f} - {interval_2param[1]:.1f} дней")
    
    # Максимальный срок с надежностью
    max_duration_2param = np_solver.max_duration_with_reliability(gamma, 'two_param')
    print(f"Максимальный срок с надежностью {gamma}: {max_duration_2param:.1f} дней")

print("\n=== СРАВНЕНИЕ РЕЗУЛЬТАТОВ ===")
if critical_3param and critical_2param:
    print(f"Разница в ожидаемом времени: {critical_2param['critical_time'] - critical_3param['critical_time']} дней")
    print(f"Разница в вероятности выполнения: {prob_2param - prob_3param:.2%}")

=== ТРЕХПАРАМЕТРИЧЕСКАЯ МОДЕЛЬ ===

Ожидаемое время и дисперсия (трехпараметрическая модель):
b1: t_ож = 5, D(x) = 1.0
b2: t_ож = 5, D(x) = 0.25
b3: t_ож = 6, D(x) = 3.36
b4: t_ож = 6, D(x) = 0.69
b5: t_ож = 5, D(x) = 0.44
b6: t_ож = 8, D(x) = 1.36
b7: t_ож = 5, D(x) = 1.0
b8: t_ож = 8, D(x) = 1.78
b9: t_ож = 6, D(x) = 1.36
b10: t_ож = 6, D(x) = 1.78
b11: t_ож = 5, D(x) = 1.36

Критический путь: ['b6']
Ожидаемое время выполнения: 8 дней
Дисперсия критического пути: 1.36
Среднеквадратическое отклонение: 1.17

Вероятность выполнения к сроку 25 дней: 100.00%
Интервал гарантированного выполнения (P=0.9973): 4.5 - 11.5 дней
Максимальный срок с надежностью 0.9: 9.5 дней

=== ДВУХПАРАМЕТРИЧЕСКАЯ МОДЕЛЬ ===

Ожидаемое время и дисперсия (двухпараметрическая модель):
b1: t*_ож = 7, D(x) = 1.0
b2: t*_ож = 6, D(x) = 0.25
b3: t*_ож = 9, D(x) = 3.36
b4: t*_ож = 6, D(x) = 0.69
b5: t*_ож = 4, D(x) = 0.44
b6: t*_ож = 7, D(x) = 1.36
b7: t*_ож = 7, D(x) = 1.0
b8: t*_ож = 10, D(x) = 1.78
b9: t*_ож = 6, D(